# TextCNN

In [6]:
# !pip install tensorflow
# !pip install imblearn

In [6]:
!pip install imblearn

  Using cached imblearn-0.0-py2.py3-none-any.whl (1.9 kB)
  Using cached imbalanced_learn-0.11.0-py3-none-any.whl (235 kB)

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip install nltk

  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


### [reference]

- Getting started with Text Processing   
https://www.kaggle.com/code/sudalairajkumar/getting-started-with-text-preprocessing

In [8]:
!pip install pandas


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [9]:
import os
import os
print(os.getcwd())  # 현재 작업 디렉토리 출력

/root/kisa-ai


In [61]:
import numpy as np
import pandas as pd
import re
import nltk
# import spacy
import string

df= pd.read_csv('../dataset/A-Track-clean.csv')
# df = pd.read_csv('A-dataset-clean.csv')
# df = pd.read_csv('/root/kisa-ai/A-dataset-clean.csv')

In [38]:
df.head()

,Unnamed: 0,payload,label_action
0,0,"get /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,"post /owa/auth/logon.aspx?replacecurrent=1"") a...",System_Cmd_Execution
2,2,get /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,get / http/1.1\r\n\r\n,HOST_Scan
4,4,get /sub_04_1_read.php?page=1&id=31);select * ...,System_Cmd_Execution


In [62]:
df.columns

Index(['Unnamed: 0', 'payload', 'label_action'], dtype='object')

In [63]:
df['label_action'].value_counts()

Vulnerability_Scan               16867
System_Cmd_Execution              9807
HOST_Scan                         6315
Path_Disclosure                   4775
SQL_Injection                     3395
Cross_Site_Scripting              1348
Automatically_Searching_Infor     1119
Leakage_Through_NW                 956
Directory_Indexing                 418
Name: label_action, dtype: int64

In [39]:
df['payload'] = df['payload'].astype(str)

#### Removal of HTML Tags

In [40]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [41]:
df.loc[:, "payload"] = df["payload"].apply(remove_html)

In [42]:
df.sample(5)

,Unnamed: 0,payload,label_action
5063,5063,get / http/1.0\r\n\r\n,HOST_Scan
41742,41742,post /admin/login.php http/1.0\r\nhost: 10.0.1...,Vulnerability_Scan
25731,25731,get /phpinfo.php3 http/1.1\r\nconnection: keep...,Path_Disclosure
28755,28755,get /goods/goods_search?display_type=list&arr_...,System_Cmd_Execution
39654,39654,post /admin/login.php http/1.0\r\nhost: 10.0.1...,Vulnerability_Scan


In [43]:
df.rename(columns={'Unnamed: 0':'Log_Number'}, inplace=True)

#### Removal of Punctuations(구두점 제거)(특수기호 같은거)

One another common text preprocessing technique is to remove the punctuations from the text data.   
This is again a text standardization process that will help to treat 'hurray' and 'hurray!' in the same way.   
We also need to carefully choose the list of punctuations to exclude depending on the use case. For example, the string.   
punctuation in python contains the following punctuation symbols   

!"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~`   

We can add or remove more punctuations as per our need.   

In [44]:


PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df['payload'] = df['payload'].apply(lambda text: remove_punctuation(text))
df.head()

,Log_Number,payload,label_action
0,0,get forum1professionnelaspnbootiniampnn100ampp...,System_Cmd_Execution
1,1,post owaauthlogonaspxreplacecurrent1 and 9294u...,System_Cmd_Execution
2,2,get goodsgoodssearchdisplaytypelistarrsearchli...,SQL_Injection
3,3,get http11rnrn,HOST_Scan
4,4,get sub041readphppage1id31select from generat...,System_Cmd_Execution


In [45]:
df.to_csv('A-Track-cleanplus.csv')

https://www.kaggle.com/code/mlwhiz/learning-text-classification-textcnn/notebook

In [82]:
import pandas as pd
import numpy as np
import re
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

In [83]:
data.head(5)

,Unnamed: 0,payload,label_action
0,0,"get /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,"post /owa/auth/logon.aspx?replacecurrent=1"") a...",System_Cmd_Execution
2,2,get /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,get / http/1.1\r\n\r\n,HOST_Scan
4,4,get /sub_04_1_read.php?page=1&id=31);select * ...,System_Cmd_Execution


In [84]:
# 요 방법도 있음

# Some preprocesssing that will be common to all the text classification methods you will see. 
puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def remove_html(x):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', x)

In [85]:
data['payload'] = data['payload'].apply(lambda x: remove_html(x))

In [86]:
data.sample(5)

,Unnamed: 0,payload,label_action
12245,12245,post /controller/login.php http/1.1\r\ncontent...,Vulnerability_Scan
42029,42029,get /company/about http/1.1\r\ncache-control: ...,Leakage_Through_NW
43916,43916,get /goods/goods_search?display_type=list&arr_...,Vulnerability_Scan
71,71,get /data/skin/responsive_ver1_default_gl/comm...,System_Cmd_Execution
13625,13625,get /dir_thatware/config.php?root_path=http://...,Vulnerability_Scan


In [87]:
# data['payload'] = data['payload'].apply(lambda x: clean_text(x))

In [88]:
data.head()

,Unnamed: 0,payload,label_action
0,0,"get /forum1_professionnel.asp?n=/.\\\""./.\\\""....",System_Cmd_Execution
1,1,"post /owa/auth/logon.aspx?replacecurrent=1"") a...",System_Cmd_Execution
2,2,get /goods/goods_search?display_type=list&arr_...,SQL_Injection
3,3,get / http/1.1\r\n\r\n,HOST_Scan
4,4,get /sub_04_1_read.php?page=1&id=31);select * ...,System_Cmd_Execution


[kaggle CNN model Architecture]   
https://www.kaggle.com/code/kadhambari/multi-class-text-classification

In [103]:
# to be imported
from keras.preprocessing.text import text_to_word_sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

# Load your data (replace with your data loading logic)
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])
num_classes = len(label_encoder.classes_)

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
X = pad_sequences(X, maxlen=201)
y = data['label_encoded'].values

# Shuffle and split the data while maintaining class balance
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Define model parameters
max_features = 1000
maxlen = 201
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250

# CNN with max pooling implementation
print('Build model...')
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding='valid', activation='relu', strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(num_classes, activation='softmax'))  # Use softmax once

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_split=0.1)

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test, y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn-kaggle-c.h5')
print('Model saved as textcnn-kaggle-c.h5')


Build model...
Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/python3.9

In [101]:
# to be imported
from keras.preprocessing.text import text_to_word_sequence
import pandas as pd
from keras.preprocessing.text import Tokenizer
import numpy as np
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D

# Load your data (replace with your data loading logic)
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
# max_sequence_length = max(len(x) for x in X)
# X = pad_sequences(X, maxlen=max_sequence_length)
X = pad_sequences(X, maxlen=201)
y = data['label_encoded'].values
num_classes = len(np.unique(y))  # 동적으로 클래스 개수를 찾음
print(num_classes)
# Shuffle and split the data while maintaining class balance
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

from tensorflow.keras.preprocessing.sequence import pad_sequences

# # 모든 시퀀스가 길이 201을 가지도록 합니다.
# X_train = pad_sequences(X_train, maxlen=201)
# X_test = pad_sequences(X_test, maxlen=201)



max_features = 1000
maxlen = 201
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250


# CNN with max pooling imeplementation 
print('Build model...')
model = Sequential()
# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(num_classes, activation='softmax'))
# model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=32,
          epochs=10,
          validation_data=(X_test, y_test))


# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test, y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn-kaggle-c.h5')
print('Model saved as textcnn-kaggle-c.h5')

9
Build model...
Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.9/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "/usr/local/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "/usr/local/lib/python3.9

### [Textcnn+Bidirectional(LSTM(64)]

In [93]:
import numpy as np
import pandas as pd
import keras
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Bidirectional, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score, classification_report

# Load your data (replace with your data loading logic)
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
max_sequence_length = max(len(x) for x in X)
X = pad_sequences(X, maxlen=max_sequence_length)
y = data['label_encoded'].values


# Shuffle and split the data while maintaining class balance
X, y = shuffle(X, y, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Model design with Conv1D and Bidirectional LSTM
embedding_dim = 100
filter_sizes = [3, 4, 5]
num_filters = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))

conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='relu')(model.layers[-1].output)
    pool_layer = MaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))  # Adding a Bidirectional LSTM layer
model.add(Bidirectional(LSTM(64)))  # Adding another Bidirectional LSTM layer

model.add(Dense(data['label_encoded'].nunique(), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test, y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn-lstm.h5')
print('Model saved as textcnn-lstm.h5')


ValueError: Input 0 of layer "bidirectional_2" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 128600)

In [89]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle  # Import shuffle for balanced splitting
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
# data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path
# data = pd.read_csv('A-Track-cleanplus.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
max_sequence_length = max(len(x) for x in X)
X = pad_sequences(X, maxlen=max_sequence_length)
y = data['label_encoded'].values  # Use label_encoded instead of one-hot encoding

# Split the data into training and testing sets while maintaining class balance
X, y = shuffle(X, y, random_state=42)  # Shuffle the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Model design with MaxPooling1D
embedding_dim = 100
filter_sizes = [3, 4, 5]
num_filters = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))

conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='relu')(model.layers[-1].output)
    pool_layer = MaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)

model.add(Flatten())
if len(conv_layers) > 1:
    model.add(Dense(num_filters * len(conv_layers), activation='relu'))
else:
    model.add(Dense(num_filters, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(data['label_encoded'].nunique(), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model-cleanplus.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)

y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test, y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test, y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn-cleanplus.h5')
print('Model saved as textcnn-cleanplus.h5')


Epoch 1/20
507/507 [==============================] - 182s 356ms/step - loss: 0.7737 - accuracy: 0.7234 - val_loss: 0.5241 - val_accuracy: 0.7978
Epoch 2/20
507/507 [==============================] - 179s 353ms/step - loss: 0.4442 - accuracy: 0.8287 - val_loss: 0.4458 - val_accuracy: 0.8189
Epoch 3/20
507/507 [==============================] - 179s 352ms/step - loss: 0.3394 - accuracy: 0.8695 - val_loss: 0.4143 - val_accuracy: 0.8372
Epoch 4/20
507/507 [==============================] - 175s 346ms/step - loss: 0.2743 - accuracy: 0.8929 - val_loss: 0.4493 - val_accuracy: 0.8392
Epoch 5/20
507/507 [==============================] - 175s 345ms/step - loss: 0.2378 - accuracy: 0.9076 - val_loss: 0.4629 - val_accuracy: 0.8386
Epoch 6/20
282/282 [==============================] - 8s 27ms/step - loss: 0.3987 - accuracy: 0.8443
Test accuracy: 0.8443333506584167
282/282 [==============================] - 8s 27ms/step

---- METRICS RESULTS FOR TESTING DATA ----

Total Rows are:  9000
[TESTING] Mo

## CONV1D

[reference]

https://chat.openai.com/share/99c938a2-8ef6-4930-89a9-f9cdaea92007

[try01]textcnn.h5

In [ ]:
import pandas as pd
import nuumpy as np
data = pd.read_csv('A-Track-cleanplus.csv')  # Replace with your CSV file path

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
# data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path
# data = pd.read_csv('A-Track-cleanplus.csv')  # Replace with your CSV file path
# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
X = pad_sequences(X)

y = pd.get_dummies(data['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model design with MaxPooling1D
embedding_dim = 100
filter_sizes = [2, 3, 4]
num_filters = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=X.shape[1]))

for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='relu')(model.layers[-1].output)
    pool_layer = MaxPooling1D()(conv_layer)
    model.add(Flatten())
    
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adding Dropout for regularization
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=[label_names[i] for i in range(len(label_names))])

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn.h5')
print('Model saved as textcnn.h5')


ValueError: Input 0 of layer "conv1d_24" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 16900)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
df = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path
# data = pd.read_csv('A-Track-cleanplus.csv')  # Replace with your CSV file path
# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label_action'])

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['payload'])
X = tokenizer.texts_to_sequences(df['payload'])
X = pad_sequences(X)

y = pd.get_dummies(df['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model design with Dropout
embedding_dim = 100
filter_sizes = [2, 3, 4]
num_filters = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=X.shape[1]))

for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='relu')(model.layers[-1].output)
    pool_layer = MaxPooling1D()(conv_layer)
    model.add(pool_layer)

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adding Dropout for regularization
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=[label_names[i] for i in range(len(label_names))])

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn.h5')
print('Model saved as textcnn.h5')

[try02] textcnn01.h5

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
# data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])

# Tokenize the text
max_words = 10000
maxlen = 100  # Sequence length
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
X = pad_sequences(X, maxlen=maxlen)

y = pd.get_dummies(data['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model design with Conv1D and GlobalMaxPooling1D
embedding_dim = 100
filter_sizes = [2, 3, 4]
num_filters = 128

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
for filter_size in filter_sizes:
    model.add(Conv1D(num_filters, filter_size, activation='relu'))
    model.add(MaxPooling1D())

model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn01.h5')
print('Model saved as textcnn01.h5')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
# data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label_action'])

# Tokenize the text
max_words = 10000
maxlen = 100  # Sequence length
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['payload'])
X = tokenizer.texts_to_sequences(df['payload'])
X = pad_sequences(X, maxlen=maxlen)

y = pd.get_dummies(df['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modified model architecture
embedding_dim = 100
filter_sizes = [3, 4, 5]
num_filters = 128
dropout_rate = 0.5

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))

for filter_size in filter_sizes:
    model.add(Conv1D(num_filters, filter_size, activation='relu'))
    model.add(MaxPooling1D())

model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, 
                    validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn02.h5')
print('Model saved as textcnn02.h5')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the data
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])

# Tokenize the text
max_words = 10000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
X = pad_sequences(X)

y = pd.get_dummies(data['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model design with Dropout
embedding_dim = 100
filter_sizes = [2, 3, 4]
num_filters = 128

inputs = Input(shape=(X.shape[1],))
embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=X.shape[1])(inputs)
conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(num_filters, filter_size, activation='relu')(embedding)
    pool_layer = MaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)

concat = Flatten()(conv_layers[-1]) if len(filter_sizes) > 1 else Flatten()(conv_layers[0])
concat = Dropout(0.5)(concat)
output = Dense(9, activation='softmax')(concat)

model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

epochs = 20
batch_size = 64
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('textcnn01.h5')
print('Model saved as textcnn01.h5')


### CONV2D

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import re
# Load the data
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])


# 데이터 정제 함수
def preprocess_text(text):
    # 특수문자 및 공백 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # HTML 태그 제거 (필요 시)
    text = re.sub(r'<.*?>', '', text)
    # 소문자 변환
    text = text.lower()
    return text

# Payload 열에 대해 데이터 정제
data['payload'] = data['payload'].apply(preprocess_text)

# Tokenize the text
# max_words = 10000
max_words = 1000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
X = pad_sequences(X)

y = pd.get_dummies(data['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model design with Conv2D layers
embedding_dim = 100
filter_sizes = [2, 3, 4]
num_filters = 128

inputs = Input(shape=(X.shape[1],))
embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=X.shape[1])(inputs)
conv_layers = []
for filter_size in filter_sizes:
    reshape = Reshape((X.shape[1], embedding_dim, 1))(embedding)
    conv_layer = Conv2D(num_filters, (filter_size, embedding_dim), activation='relu')(reshape)
    pool_layer = MaxPooling2D(pool_size=(X.shape[1] - filter_size + 1, 1))(conv_layer)
    conv_layers.append(pool_layer)

concat = Concatenate(axis=1)(conv_layers) if len(filter_sizes) > 1 else conv_layers[0]
flatten = Flatten()(concat)
flatten = Dropout(0.5)(flatten)
output = Dense(9, activation='softmax')(flatten)

model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# epochs = 20
epochs = 10
# batch_size = 64
batch_size = 128
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('conv2d_textcnn.h5')
print('Model saved as conv2d_textcnn.h5')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Reshape, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import re
# Load the data
data = pd.read_csv('../dataset/A-Track-clean.csv')  # Replace with your CSV file path

# Encode labels
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label_action'])


# 데이터 정제 함수
def preprocess_text(text):
    # 특수문자 및 공백 제거
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # HTML 태그 제거 (필요 시)
    text = re.sub(r'<.*?>', '', text)
    # 소문자 변환
    text = text.lower()
    return text

# Payload 열에 대해 데이터 정제
data['payload'] = data['payload'].apply(preprocess_text)

# Tokenize the text
# max_words = 10000
max_words = 1000
tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')  # 구두점 제거 필터 추가
tokenizer.fit_on_texts(data['payload'])
X = tokenizer.texts_to_sequences(data['payload'])
X = pad_sequences(X)

y = pd.get_dummies(data['label_encoded']).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model design with Conv2D layers
embedding_dim = 100
filter_sizes = [2, 3, 4]
num_filters = 128

inputs = Input(shape=(X.shape[1],))
embedding = Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=X.shape[1])(inputs)
conv_layers = []
for filter_size in filter_sizes:
    reshape = Reshape((X.shape[1], embedding_dim, 1))(embedding)
    conv_layer = Conv2D(num_filters, (filter_size, embedding_dim), activation='relu')(reshape)
    pool_layer = MaxPooling2D(pool_size=(X.shape[1] - filter_size + 1, 1))(conv_layer)
    conv_layers.append(pool_layer)

concat = Concatenate(axis=1)(conv_layers) if len(filter_sizes) > 1 else conv_layers[0]
flatten = Flatten()(concat)
flatten = Dropout(0.5)(flatten)
output = Dense(9, activation='softmax')(flatten)

model = Model(inputs=inputs, outputs=output)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training with EarlyStopping and ModelCheckpoint
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True)

# epochs = 20
epochs = 10
# batch_size = 64
batch_size = 128
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[early_stop, model_checkpoint])

# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print('Test accuracy:', accuracy)
y_pred_test = model.predict(X_test)
test_acc = accuracy_score(y_test.argmax(axis=1), y_pred_test.argmax(axis=1))
test_rep = classification_report(y_test.argmax(axis=1), y_pred_test.argmax(axis=1), target_names=label_encoder.classes_)

print()
print("---- METRICS RESULTS FOR TESTING DATA ----")
print()
print("Total Rows are: ", X_test.shape[0])
print('[TESTING] Model Accuracy is: ', test_acc)
print('[TESTING] Testing Report: ')
print(test_rep)

# Save the model
model.save('conv2d_textcnn.h5')
print('Model saved as conv2d_textcnn.h5')


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences

from imblearn.over_sampling import SMOTE

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# 데이터 세트 준비
df = pd.read_csv('../dataset/A-Track-clean.csv')

# 데이터 전처리
x_train = df['payload'].values
y_train = df['label_action'].values

# 데이터의 불균형 조정
oversample = SMOTE()
x_train, y_train = oversample.fit_resample(x_train, y_train)

# 텍스트 토큰화
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(x_train)

# 텍스트 패딩
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, padding='post', maxlen=100)

# 모델 생성
model = Sequential()
model.add(Embedding(input_dim=10000, output_dim=128))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(9, activation='softmax'))

# 모델 학습
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=128)

# 모델 평가
score = model.evaluate(x_train, y_train, verbose=0)
print('Accuracy:', score[1])


2023-10-18 17:55:10.193948: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 17:55:10.651559: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2023-10-18 17:55:10.651688: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2023-10-18 17:55:10.755205: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


ModuleNotFoundError: No module named 'imblearn'